In [1]:
import pandas as pd
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, ColumnDataSource, HoverTool

import sys
sys.path.append("../")
from Code.visualisation import visualise_configuration, use_cases
from Code.helper_functions import initial_loading, analyze_candidate_solution, add_shortest_distances_to_all_households

POSTGRES password?WelkomCorne!


In [ ]:
file = gpd.read_file('../Data/shp/Inzameling_huisvuil_080520.shp')
file_clean = file[file['aanbiedwij'] == 'Breng uw restafval  naar een container voor restafval.']
wijken = gpd.read_file('../data/brtk2010_ind2005_region.shp')
file['aanbiedwij'].value_counts()

In [ ]:
geosource = GeoJSONDataSource(geojson=file.to_json())
geosource2= GeoJSONDataSource(geojson=file_clean.to_json())
geosource3= GeoJSONDataSource(geojson=wijken.to_json())

In [ ]:
p = figure(match_aspect=True)
p.patches('xs', 'ys', source=geosource, fill_color='grey', alpha=0.1, line_color=None)
p.patches('xs', 'ys', source=geosource2, fill_color='grey', alpha=0.1, line_color=None)
p.patches('xs', 'ys', source=geosource3, fill_color=None, alpha=0.1, line_color='blue')

show(p)

## Use case

In [ ]:
def visualise_configuration():
    """
    Create Bokeh Plot of candidate solution.

    Function prompts for filename to be plotted. It then loads in the current
    situation for comparison purposes. It plots the garbage container clusters
    and active addresses on a shapefile of the city. All places were garbage is
    centrally collected is marked in darker grey. The Bokeh Plot contains an
    interactive HoverTool that shows the compoisitions of either the houses or
    garbage containers.
    """
    filename = input("Please provide the requested filename")
    area = input("Provide Code of neighborhood (T/N/F/C)")
    if area == 'C':
        area = ['M', 'A', 'K', 'E']
    df_zo = pd.read_csv(filename)

    all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

    to_join = df_zo[['s1_afv_nodes', 'rest', 'plastic', 'papier', 'glas',
                     'textiel', 'totaal']].set_index('s1_afv_nodes')
    joined.s1_afv_nodes = joined.s1_afv_nodes.astype('int')
    new = to_join.join(joined.set_index('s1_afv_nodes'), lsuffix='_n')

    households_zo = all_households[all_households['in_neigborhood']]
    households_zo = households_zo.set_index('naar_s1_afv_nodes')\
        .join(df_afstandn2[['count', 'naar_s1_afv_nodes']]
              .set_index('naar_s1_afv_nodes'), how='left').drop_duplicates()
    households_zo = households_zo[households_zo['count'] > 0]


    load = gpd.read_file('../data/Inzameling_huisvuil_100220.shp')

    street_map = load[load['sdcode'].isin(list(area))]
    geosource = GeoJSONDataSource(geojson=street_map.to_json())
    street_map_clean = street_map[street_map['aanbiedwij'] ==
                                  'Breng uw restafval  naar een container voor restafval.']
    geosource2 = GeoJSONDataSource(geojson=street_map_clean.to_json())

    TOOLTIPS = [
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("Rest", "(@rest, @rest_n)"),
                ("Plastic", "(@plastic, @plastic_n)"),
                ("Papier", "(@papier, @papier_n)"),
                ("Glas", "(@glas, @glas_n)"),
                ("Textiel", "(@textiel, @textiel_n)")
                ]

    TOOLTIPS2 = [
                ("index", "$index"),
                ("(x,y)", "($x, $y)"),
                ("Count", "@count")]

    source1 = ColumnDataSource(data=new
                               [['cluster_x', 'cluster_y', 'rest', 'papier',
                                 'plastic', 'textiel', 'glas', 'rest_n',
                                 'papier_n', 'glas_n', 'plastic_n',
                                 'textiel_n']])
    source2 = ColumnDataSource(data=households_zo)

    p = figure(match_aspect=True)
    p.patches('xs', 'ys', source=geosource, fill_color='grey', alpha=0.1,
              line_color=None)
    p.patches('xs', 'ys', source=geosource2, fill_color='grey', alpha=0.3,
              line_color=None)
    r3 = p.circle(x='cluster_x', y='cluster_y', color='red', line_color=None,
                  source=source1, radius=20)
    p.add_tools(HoverTool(renderers=[r3], tooltips=TOOLTIPS))
    r4 = p.circle(x='cluster_x', y='cluster_y', radius=1.5, source=source2)
    p.add_tools(HoverTool(renderers=[r4], tooltips=TOOLTIPS2))

    show(p)
    return p, df_zo, households_zo.reset_index().rename(columns={'index': 'naar_s1_afv_nodes'}), all_households, rel_poi_df, joined, df_afstandn2

p, containers, households, all_households, rel_poi_df, joined, df_afstandn2 = visualise_configuration()

In [ ]:
all_households, rel_poi_df, joined, df_afstandn2 = initial_loading()

In [ ]:
# joined_cluster_distance, good_result, aansluitingen,\
#         avg_distance, penalties, simple_penalties = analyze_candidate_solution(
#     joined, households, rel_poi_df, df_afstandn2, use_count=True, clean=False)

joined_cluster_distance = containers.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

# Add shortest distances to it
good_result = \
    add_shortest_distances_to_all_households(households,
                                             joined_cluster_distance)

# good_result = good_result[good_result['count'] > 0]
good_result = good_result.reset_index()

In [ ]:
good_result[good_result.index == 6248]

In [ ]:
# joined_cluster_distance, good_result, aansluitingen,\
#         avg_distance, penalties, simple_penalties = analyze_candidate_solution(
#     containers, all_households, rel_poi_df, df_afstandn2, use_count=True, clean=False)

joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

# Add shortest distances to it
good_result = \
    add_shortest_distances_to_all_households(all_households,
                                             joined_cluster_distance,
                                             use_count=True)

good_result = good_result.reset_index()

In [ ]:
good_result[good_result['naar_s1_afv_nodes'] == 748684]

In [ ]:
def use_cases(indx):
    p, containers, households, all_households, rel_poi_df, joined, df_afstandn2 = visualise_configuration()    
    
    # Collect New
    joined_cluster_distance = containers.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

    # Add shortest distances to it
    good_result = \
        add_shortest_distances_to_all_households(households,
                                                 joined_cluster_distance)

    # good_result = good_result[good_result['count'] > 0]
    good_result = good_result.reset_index()
    print(good_result[good_result.index == indx])
    poi = int(good_result[good_result.index == indx]['naar_s1_afv_nodes'])
    
    
    # Collect old
    joined_cluster_distance = joined.set_index('s1_afv_nodes')\
    .join(df_afstandn2.set_index('van_s1_afv_nodes')).reset_index()\
    .rename(columns={'index': 'van_s1_afv_nodes'})

    # Add shortest distances to it
    good_result = \
        add_shortest_distances_to_all_households(all_households,
                                             joined_cluster_distance,
                                             use_count=True)
    
    good_result = good_result.reset_index().rename(columns={'index': 'naar_s1_afv_nodes'})
    print(good_result[good_result['naar_s1_afv_nodes'] == poi])
    return

In [2]:
use_cases(6248)

Please provide the requested filename../Results/hillclimber_best_config2020-05-04-10ZO.csv
Provide Code of neighborhood (T/N/F/C)T
Do you want to use addresses instead of clusters?True
What stadsdeel do you want to make as a subsection(optional parameter)?T
What is the maximum amount of containers in a cluster that is considered to be useful?8
Where to get db files(local/online)?local
DB relation POIs loaded


..\Code\loading_data.py:389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['ligtin_bag_pnd_identificatie'] = \
..\Code\loading_data.py:397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  verblijfsobjecten['bag'] = verblijfsobjecten.loc[:, 'bag']\


distance matrix loaded
API data loaded
Table all households created
API and DB joined
containers per cluster determined
move_rest determined
      naar_s1_afv_nodes  cluster_x  cluster_y uses_container  \
6248             748684   127539.0   480836.0          False   

     collect_cardboard in_landelijk_noord in_centrum in_neigborhood  count  \
6248             False              False      False           True    1.0   

      poi_rest  rest_afstand  poi_plastic  plastic_afstand  poi_papier  \
6248  572311.0        161.74     683743.0           262.86    889308.0   

      papier_afstand  poi_glas  glas_afstand  poi_textiel  textiel_afstand  
6248          631.15  874450.0         37.61     489362.0           968.37  
       naar_s1_afv_nodes  cluster_x  cluster_y  uses_container  \
88575             748684     127539     480836           False   

       collect_cardboard  in_landelijk_noord  in_centrum  in_neigborhood  \
88575              False               False       False     

In [ ]:
poi = 748684
good_result = \
    add_shortest_distances_to_all_households(all_households,
                                         joined_cluster_distance,
                                         use_count=True)
good_result = good_result.reset_index().rename(columns={'index': 'naar_s1_afv_nodes'})
print(good_result[good_result['naar_s1_afv_nodes'] == poi])